In [1]:
from pprint import pprint

import torch
import torch.nn as nn
import torch.onnx


class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 4, True)
        self.relu = nn.ReLU()
    def forward(self, x):
        return self.relu(self.linear(x))
model = TestModel()
dummy_input = torch.zeros(8)
torch.onnx.export(model, dummy_input, "singlelayer.onnx", verbose=True)

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [2]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("singlelayer.onnx")
onnx_model = onnx.shape_inference .infer_shapes(onnx_model)
# Check that the model is well formed
onnx.checker.check_model(onnx_model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %onnx::MatMul_0[FLOAT, 8]
) initializers (
  %linear.bias[FLOAT, 4]
  %onnx::MatMul_7[FLOAT, 8x4]
) {
  %/linear/MatMul_output_0 = MatMul(%onnx::MatMul_0, %onnx::MatMul_7)
  %/linear/Add_output_0 = Add(%linear.bias, %/linear/MatMul_output_0)
  %6 = Relu(%/linear/Add_output_0)
  return %6
}


In [3]:
for init in onnx_model.graph.initializer:
    print(init)
for node in onnx_model.graph.node:
    print(node)

dims: 4
data_type: 1
name: "linear.bias"
raw_data: "@\266\026\274J\206u>\314\372\246\275a\204\244\276"

dims: 8
dims: 4
data_type: 1
name: "onnx::MatMul_7"
raw_data: "\n\233\227\276\215\333\241>\203\360\224>:\323\242\276\243&\260\276\371\201\241>\274\333\027\276N\236_\276\013\246Q\276\371\323\242>Y\264\232>E0\223>1Z\257>\001\334\227>\n\263q>\345\334\222\276 \201\021>8u9=\253g\262>\3605\272<\213\246\224\276x\354\346===\225\2760\303&=\304E\237\276\017e\256>\322ch>0\026\345<V\214]>\021\030\217>@\377\304\275\212\365\007\276"

input: "onnx::MatMul_0"
input: "onnx::MatMul_7"
output: "/linear/MatMul_output_0"
name: "/linear/MatMul"
op_type: "MatMul"
doc_string: "C:\\Users\\TheoA\\AppData\\Roaming\\Python\\Python39\\site-packages\\torch\\nn\\modules\\linear.py(114): forward\nC:\\Users\\TheoA\\AppData\\Roaming\\Python\\Python39\\site-packages\\torch\\nn\\modules\\module.py(1488): _slow_forward\nC:\\Users\\TheoA\\AppData\\Roaming\\Python\\Python39\\site-packages\\torch\\nn\\modules\\module.py(15

In [4]:
init_node = onnx_model.graph.node[0]
init_node.input[1]
onnx_model.graph.initializer
next(i for i in iter(onnx_model.graph.initializer) if i.name == "onnx::MatMul_7").dims

[8, 4]

In [5]:
from compiler import parsemodel, fpgamodule
spec = fpgamodule.FPGASpec(120, 1000, 10_000, 100_000)
fpga_module = parsemodel.parse_model(onnx_model, 8, 4, spec)
pprint(list(mod for mod in fpga_module.modules))

[vecfifo_0:[0, wr_data_0, rd_data_0, wr_en_0, rd_en_0, vecfifo_0_out_vec_valid_0, wrap_rd_0],
 mvprod_0:[in_data_ready_master, rd_data_0, wr_data_1, rd_en_0, wr_en_1, mvprod_0_out_vec_valid_0, wrap_rd_0],
 vecfifo_1:[0, wr_data_1, rd_data_1, wr_en_1, rd_en_1, vecfifo_1_out_vec_valid_1, 0],
 bias_0:[mvprod_0_out_vec_valid_0, rd_data_1, wr_data_2, rd_en_1, wr_en_2, bias_0_out_vec_valid_0],
 vecfifo_2:[0, wr_data_2, rd_data_2, wr_en_2, rd_en_2, vecfifo_2_out_vec_valid_2, 0],
 relu_0:[bias_0_out_vec_valid_0, rd_data_2, wr_data_3, rd_en_2, wr_en_3, relu_0_out_vec_valid_0],
 vecfifo_3:[0, wr_data_3, rd_data_3, wr_en_3, rd_en_3, vecfifo_3_out_vec_valid_3, 0]]


In [6]:
fpga_module.alloc_regs()
fpga_module.alloc_bram()
sv = fpga_module.make_sv()
with open("dummy_model.sv", "w") as f:
    f.write(sv)